In [ ]:
#pip install geopandas

In [ ]:
#pip install folium matplotlib mapclassify

In [88]:
import warnings
warnings.filterwarnings('ignore')

In [38]:
import geopandas as gpd 
import matplotlib.pyplot as plt
import numpy as np
from math import pi, sqrt
from matplotlib.patches import ConnectionPatch

In [2]:
df = gpd.read_file("BF_10_03_2020.zip")

In [3]:
df2 = gpd.read_file("BF_08_02_2020.zip")

In [4]:
df2.head()

,mvt_id,DN,geometry
0,None,4.0,"POLYGON ((-12386072.825 3999537.731, -12386072..."
1,None,4.0,"POLYGON ((-12386070.156 3999538.766, -12386070..."
2,None,4.0,"POLYGON ((-12386067.954 3999531.946, -12386067..."
3,None,4.0,"POLYGON ((-12386071.341 3999548.349, -12386071..."
4,None,4.0,"POLYGON ((-12386070.595 3999548.302, -12386070..."


# For df1 (Oct 2020) 

In [5]:
df["area"] = df.area

In [6]:
df['centroid'] = df.centroid
df['centroid']

0       POINT (-12386059.874 3999557.488)
1       POINT (-12386059.123 3999554.551)
2       POINT (-12386059.490 3999554.273)
3       POINT (-12386058.413 3999557.855)
4       POINT (-12386057.928 3999557.859)
                      ...                
7360    POINT (-12385927.687 3999598.412)
7361    POINT (-12385920.748 3999494.777)
7362    POINT (-12385922.418 3999593.478)
7363    POINT (-12385922.833 3999591.188)
7364    POINT (-12385921.579 3999589.459)
Name: centroid, Length: 7365, dtype: geometry

In [ ]:
plt.figure(figsize=(16, 8))
df["area"].plot(kind='hist', bins = 100)

In [ ]:
# plt.figure(figsize=(16, 16))
df.plot("area", legend=True, figsize=(16, 16))

In [ ]:
# plt.figure(figsize=(16, 16))
df = df.set_geometry("centroid")
df.plot("area", legend=True, figsize=(16, 16))

In [8]:
df = df.set_geometry("geometry")

# For df2 (Aug 2020)

In [7]:
df2["area"] = df2.area

In [8]:
df2['centroid'] = df2.centroid
df2['centroid']

0       POINT (-12386072.636 3999537.542)
1       POINT (-12386070.184 3999538.509)
2       POINT (-12386068.133 3999531.497)
3       POINT (-12386071.411 3999548.372)
4       POINT (-12386070.759 3999548.336)
                      ...                
3666    POINT (-12385914.798 3999616.488)
3667    POINT (-12385914.654 3999615.601)
3668    POINT (-12385912.517 3999528.752)
3669    POINT (-12385912.519 3999539.630)
3670    POINT (-12385913.220 3999609.211)
Name: centroid, Length: 3671, dtype: geometry

In [ ]:
plt.figure(figsize=(16, 8))
df2["area"].plot(kind='hist', bins = 100)

In [ ]:
df2.plot("area", legend=True, figsize=(16, 16))

## Calculating Ecc

In [9]:
## Get Eccentricity 

import numpy as np
from scipy.spatial.distance import euclidean

def get_ecc(area_polygon):
    points = np.array(area_polygon)

    small_latwise = np.min(points[points[:, 0] == np.min(points[:, 0])], 0)
    small_lonwise = np.min(points[points[:, 1] == np.min(points[:, 1])], 0)
    big_latwise = np.max(points[points[:, 0] == np.max(points[:, 0])], 0)
    big_lonwise = np.max(points[points[:, 1] == np.max(points[:, 1])], 0)
    distance_lat = euclidean(big_latwise, small_latwise)
    distance_lon = euclidean(big_lonwise, small_lonwise)
    if distance_lat >= distance_lon:
        major_axis_length = distance_lat
        minor_axis_length = distance_lon
    else:
        major_axis_length = distance_lon
        minor_axis_length = distance_lat
    a = major_axis_length/2
    b = minor_axis_length/2
    ecc = np.sqrt(np.square(a)-np.square(b))/a
    return ecc

In [ ]:
df['ecc'] = None

for j in range(len(df)):
    try:
        area_polygon = [[i[1],i[0]] for i in list(df['geometry'][j].exterior.coords)]
    except:
        area_polygon = [[i[1],i[0]]  for poly in list(df['geometry'][j]) for i in list(poly.exterior.coords) ]
    df['ecc'][j] = get_ecc(area_polygon)

In [ ]:
df2['ecc'] = None

for j in range(len(df2)):
    #print(j)
    try:
        area_polygon = [[i[1],i[0]] for i in list(df2['geometry'][j].exterior.coords)]
    except:
        area_polygon = [[i[1],i[0]]  for poly in list(df2['geometry'][j]) for i in list(poly.exterior.coords)]
            
    df2['ecc'][j] = get_ecc(area_polygon)

## TS_SS Similarity 

In [142]:
import math

def Cosine(vec1, vec2) :
    result = InnerProduct(vec1,vec2) / (VectorSize(vec1) * VectorSize(vec2))
    return result

def VectorSize(vec) :
    return math.sqrt(sum(math.pow(v,2) for v in vec))

def InnerProduct(vec1, vec2) :
    return sum(v1*v2 for v1,v2 in zip(vec1,vec2))

def Euclidean(vec1, vec2) :
    return math.sqrt(sum(math.pow((v1-v2),2) for v1,v2 in zip(vec1, vec2)))

def Theta(vec1, vec2) :
    return math.acos(Cosine(vec1,vec2)) + math.radians(10)

def Triangle(vec1, vec2) :
    theta = math.radians(Theta(vec1,vec2))
    return (VectorSize(vec1) * VectorSize(vec2) * math.sin(theta)) / 2

def Magnitude_Difference(vec1, vec2) :
    return abs(VectorSize(vec1) - VectorSize(vec2))

def Sector(vec1, vec2) :
    ED = Euclidean(vec1, vec2)
    MD = Magnitude_Difference(vec1, vec2)
    theta = Theta(vec1, vec2)
    return math.pi * math.pow((ED+MD),2) * theta/360

def TS_SS(vec1, vec2) :
    return Triangle(vec1, vec2) * Sector(vec1, vec2)

vec1 = [1,2]
vec2 = [2,4]

print(Euclidean(vec1,vec2))
print(Cosine(vec1,vec2))
print(TS_SS(vec1,vec2))

2.23606797749979
0.9999999999999998
0.0004639582566999478


# Calculating Distance and Area Distribution of neighbors

In [ ]:
df.columns 

In [ ]:
df2.columns 

In [ ]:
def get_dist_and_area_distribution(df, top=10):

  x = df.geometry.apply(lambda g: df.distance(g))
  df['dist_distribution'] = None
  df['area_distribution'] = None 
  df['neighbour_indexes'] = None
  df['ecc_distribution'] = None
  for i in range(len(df)):
    y = x[i].sort_values()[:top]
    df['dist_distribution'][i] = list(y)
    df['area_distribution'][i] = [df['area'][j] for j in y.index]
    df['ecc_distribution'][i] = [df['ecc'][j] for j in y.index]
    df['neighbour_indexes'][i] = y.index 

  return df

In [ ]:
df = get_dist_and_area_distribution(df, 10)

In [ ]:
df2 = get_dist_and_area_distribution(df2, 10)

### Getting Means of neigbors stats 

In [ ]:
## Averages of Neighbour Distance, Area and Eccentricity distributions 
def get_means(df):
    df['area_mean'] = None
    df['dist_mean'] = None 
    df['ecc_mean'] = None 

    for j in range(len(df)):

        df['area_mean'][j] = np.array(df['area_distribution'][j]).mean()
        df['dist_mean'][j] = np.array(df['dist_distribution'][j]).mean() 
        df['ecc_mean'][j] = np.array(df['ecc_distribution'][j]).mean() 
        
    return df 

In [ ]:
df = get_means(df) 

In [ ]:
df2 = get_means(df2) 

## Second Order Stats 

In [ ]:
def get_2_order_stats(df):
    df['neighbor_stats_dist'] = None 
    df['neighbor_stats_area'] = None 
    df['neighbor_stats_ecc'] = None 

    for i in range(len(df)):

        df['neighbor_stats_dist'][i] = [np.array(df['dist_distribution'][j]).mean() for j in df['neighbour_indexes'][i]]
        df['neighbor_stats_area'][i] = [np.array(df['area_distribution'][j]).mean() for j in df['neighbour_indexes'][i]]
        df['neighbor_stats_ecc'][i] = [np.array(df['ecc_distribution'][j]).mean() for j in df['neighbour_indexes'][i]]
        
    return df 

In [ ]:
df = get_2_order_stats(df)
df2 = get_2_order_stats(df2)

In [ ]:
df2.columns

In [ ]:
df.columns

In [ ]:
plt.hist(df['neighbor_stats_area'][100])
# df2['area_distribution'][0][0]

In [ ]:
plt.hist(df2['neighbor_stats_area'][100])

## TS_SS similarity based rock matching 

In [ ]:
df2.columns

In [ ]:
df['nearest_matching_polygon_wrt_dist_ts_ss'] = None
df['nearest_matching_polygon_wrt_area_ts_ss'] = None
df['nearest_matching_polygon_wrt_ecc_ts_ss'] = None
df['nearest_matching_polygon_overall_ts_ss'] = None

df['min_ts_ss_dist'] = 10000000.0
df['min_ts_ss_area'] = 10000000.0
df['min_ts_ss_ecc'] = 10000000.0
df['min_ts_ss_overall'] = 10000000.0


for i in range(len(df)):

  for j in range(len(df2)):

    ts_ss_ij_dist = TS_SS(df['dist_distribution'][i], df2['dist_distribution'][j])
    ts_ss_ij_area = TS_SS(df['area_distribution'][i], df2['area_distribution'][j])
    ts_ss_ij_ecc = TS_SS(df['ecc_distribution'][i], df2['ecc_distribution'][j])
    
    ts_ss_ij_overall = ts_ss_ij_dist + ts_ss_ij_area + ts_ss_ij_ecc

    if(df['min_ts_ss_dist'][i] > ts_ss_ij_dist):
        df['min_ts_ss_dist'][i] = ts_ss_ij_dist
        df['nearest_matching_polygon_wrt_dist_ts_ss'][i] = j
        
    if(df['min_ts_ss_area'][i] > ts_ss_ij_area):
        df['min_ts_ss_area'][i] = ts_ss_ij_area
        df['nearest_matching_polygon_wrt_area_ts_ss'][i] = j
    
    if(df['min_ts_ss_ecc'][i] > ts_ss_ij_ecc):
        df['min_ts_ss_ecc'][i] = ts_ss_ij_ecc
        df['nearest_matching_polygon_wrt_ecc_ts_ss'][i] = j

    
    if(df['min_ts_ss_overall'][i] > ts_ss_ij_overall):
        df['min_ts_ss_overall'][i] = ts_ss_ij_overall
        df['nearest_matching_polygon_overall_ts_ss'][i] = j
        

In [ ]:
df['nearest_matching_polygon_wrt_dist_ts_ss'].plot(kind='hist')

In [ ]:
df['min_ts_ss_ecc'].plot(kind='hist')

## Distribution Based Rock Matching (RMSE)

In [ ]:
def rmse(a, b):
  a = np.array(a)
  b = np.array(b)
  return np.sqrt(((a - b) ** 2).mean())

In [ ]:
df['nearest_matching_polygon_wrt_dist'] = None
df['nearest_matching_polygon_wrt_area'] = None
df['nearest_matching_polygon_overall'] = None


for i in range(len(df)):

  min_rmse_dist = 100000
  min_rmse_area = 100000
  min_rmse_overall = 100000

  for j in range(len(df2)):

    rmse_ij_dist = rmse(df['dist_distribution'][i], df2['dist_distribution'][j])
    rmse_ij_area = rmse(df['area_distribution'][i], df2['area_distribution'][j])
    rmse_ij_overall = rmse_ij_area + rmse_ij_dist 

    if(min_rmse_dist > rmse_ij_dist):
      min_rmse_dist = rmse_ij_dist
      df['nearest_matching_polygon_wrt_dist'][i] = j

    if(min_rmse_area > rmse_ij_area):
      min_rmse_area = rmse_ij_area
      df['nearest_matching_polygon_wrt_area'][i] = j

    if(min_rmse_overall > rmse_ij_overall):
      min_rmse_overall = rmse_ij_overall
      df['nearest_matching_polygon_overall'][i] = j

## Mean based rock Matching (Nearest in 3D Eucledian space) 

In [ ]:
df['nearest_matching_polygon_wrt_3d_means_ts_ss'] = None
df['min_ts_ss_3d'] = 10000000.0

for i in range(len(df)):

#   min_3d_dist = 100000000000

  for j in range(len(df2)):

    dist_3d = TS_SS([df['area_mean'][i], df['dist_mean'][i], df['ecc_mean'][i]], [df2['area_mean'][j], df2['dist_mean'][j], df2['ecc_mean'][j]]) 
    
    if df['min_ts_ss_3d'][i] > dist_3d:
        df['min_ts_ss_3d'][i] = dist_3d
        df['nearest_matching_polygon_wrt_3d_means_ts_ss'][i] = j

In [122]:
df['min_ts_ss_3d']

0       1.654948e-08
1       2.445838e-08
2       1.016930e-07
3       5.923373e-09
4       4.416877e-09
            ...     
7360    8.615610e-09
7361    1.998271e-05
7362    3.732394e-09
7363    3.268173e-09
7364    7.695708e-09
Name: min_ts_ss_3d, Length: 7365, dtype: float64

## Wasserstein Metric Based Rock Matching 

In [ ]:
from scipy import stats 

In [ ]:
df['nearest_matching_polygon_wrt_dist_was_2o'] = None
df['nearest_matching_polygon_wrt_area_was_2o'] = None
df['nearest_matching_polygon_wrt_ecc_was_2o'] = None
df['nearest_matching_polygon_overall_was_2o'] = None

df['min_was_dist_2o'] = 1000000.0
df['min_was_area_2o'] = 1000000.0
df['min_was_ecc_2o'] = 1000000.0
df['min_was_overall_2o'] = 1000000.0

weight = [1 for i in range(50)]
weight[0] = 1.5


for i in range(len(df)):

  for j in range(len(df2)):
        
    was_ij_dist = stats.wasserstein_distance(df['neighbor_stats_dist'][i], df2['neighbor_stats_dist'][j])
    was_ij_area = stats.wasserstein_distance(df['neighbor_stats_area'][i], df2['neighbor_stats_area'][j])
    was_ij_ecc = stats.wasserstein_distance(df['neighbor_stats_ecc'][i], df2['neighbor_stats_ecc'][j])
    
    was_ij_overall = was_ij_area + was_ij_dist + was_ij_ecc 

    if(df['min_was_dist_2o'][i] > was_ij_dist):
      df['min_was_dist_2o'][i] = was_ij_dist
      df['nearest_matching_polygon_wrt_dist_was_2o'][i] = j

    if(df['min_was_area_2o'][i] > was_ij_area):
      df['min_was_area_2o'][i] = was_ij_area
      df['nearest_matching_polygon_wrt_area_was_2o'][i] = j
        
    if(df['min_was_ecc_2o'][i] > was_ij_ecc):
      df['min_was_ecc_2o'][i] = was_ij_ecc
      df['nearest_matching_polygon_wrt_ecc_was_2o'][i] = j

    if(df['min_was_overall_2o'][i] > was_ij_overall):
      df['min_was_overall_2o'][i] = was_ij_overall
      df['nearest_matching_polygon_overall_was_2o'][i] = j


In [ ]:
stats.wasserstein_distance(df['dist_distribution'][1], df2['dist_distribution'][5])

In [ ]:
df['min_was_dist'][0]

In [ ]:
df[['min_was_dist', 'nearest_matching_polygon_wrt_dist_was']]

In [ ]:
idx = -1
min_was = 1000000

for i in range(len(df)):
    
#     was_now = stats.wasserstein_distance(df['neighbor_stats_area'][i], df2['neighbor_stats_area'][df['nearest_matching_polygon_wrt_area_was_2o'][i]])
    
    if min_was > df['min_was_ecc'][i]:
        min_was = df['min_was_ecc'][i]
        idx = i

In [ ]:
idx

In [ ]:
min_was

In [ ]:
#WRT DF2 and second order

df2['nearest_matching_polygon_wrt_dist_was'] = None
df2['nearest_matching_polygon_wrt_area_was'] = None
df2['nearest_matching_polygon_wrt_ecc_was'] = None
df2['nearest_matching_polygon_overall_was'] = None
df2['min_was_dist'] = 1000000
df2['min_was_area'] = 1000000
df2['min_was_ecc'] = 1000000
df2['min_was_overall'] = 1000000


for i in range(len(df2)):

#   min_was_dist = 1000000
#   min_was_area = 1000000
#   min_was_ecc = 1000000  
#   min_was_overall = 1000000

  for j in range(len(df)):

    was_ij_dist = stats.wasserstein_distance(df2['neighbor_stats_dist'][i], df['neighbor_stats_dist'][j])
    was_ij_area = stats.wasserstein_distance(df2['neighbor_stats_area'][i], df['neighbor_stats_area'][j])
    was_ij_ecc = stats.wasserstein_distance(df2['neighbor_stats_ecc'][i], df['neighbor_stats_ecc'][j])
    
    was_ij_overall = was_ij_area + was_ij_dist + was_ij_ecc 

    if(df2['min_was_dist'][i] > was_ij_dist):
      df2['min_was_dist'][i] = was_ij_dist
      df2['nearest_matching_polygon_wrt_dist_was'][i] = j

    if(df2['min_was_area'][i] > was_ij_area):
      df2['min_was_area'][i] = was_ij_area
      df2['nearest_matching_polygon_wrt_area_was'][i] = j
        
    if(df2['min_was_ecc'][i] > was_ij_ecc):
      df2['min_was_ecc'][i] = was_ij_ecc
      df2['nearest_matching_polygon_wrt_ecc_was'][i] = j

    if(df2['min_was_overall'][i] > was_ij_overall):
      df2['min_was_overall'][i] = was_ij_overall
      df2['nearest_matching_polygon_overall_was'][i] = j



In [ ]:
df2.columns

In [ ]:
df['nearest_matching_polygon_wrt_dist_was'].plot(kind='hist')

In [ ]:
df_match = df[df['nearest_matching_polygon_wrt_ecc_was']==df['nearest_matching_polygon_wrt_area_was']]
df_match.reset_index(inplace=True)
df_match 

In [ ]:
df_match = df

In [ ]:

for i in range(idx, idx+1):

  row_num = i
  match1 = gpd.GeoDataFrame(geometry=[df_match['geometry'][row_num]])
  match1["match"] = df2['geometry'][df_match['nearest_matching_polygon_wrt_area_was_2o'][row_num]]

  # print(i)

  fig, axis = plt.subplots(1, 2, figsize=(16,8))
  fig.text(0, 0, str(i))
  # fig.figure(figsize=(16,8))

  match1 = match1.set_geometry('geometry')
  # match1['area'] = match1.area
  match1.plot(legend=True, figsize=(8, 8), ax=axis[0])

  match1 = match1.set_geometry('match')
  # match1['area'] = match1.area
  match1.plot(legend=True, figsize=(8, 8), ax=axis[1])

  fig.show() 

In [ ]:
df['nearest_matching_polygon_wrt_ecc_was'].plot(kind='hist')

In [ ]:
row_num = 0 
match1 = gpd.GeoDataFrame(geometry=[df_match['geometry'][row_num]])

match1= match1.append(gpd.GeoDataFrame(geometry=[df2['geometry'][df_match['nearest_matching_polygon_overall'][row_num]]]))
match1['code'] = row_num
# match1['code'] = row_num  


In [ ]:
match1

In [ ]:
for i in range(1, len(df_match)):

  row_num = i
  match_temp = gpd.GeoDataFrame(geometry=[df_match['geometry'][row_num]])
  match_temp = match_temp.append(gpd.GeoDataFrame(geometry=[df2['geometry'][df_match['nearest_matching_polygon_overall'][row_num]]]))
  match_temp['code'] = row_num

  match1 = match1.append(match_temp)


In [ ]:
# df['match_centroid'] = None 
df['match_centroid_area'] = None 
df['match_centroid_dist'] = None 
df['match_centroid_ecc'] = None
df['match_centroid_overall'] = None

for i in range(len(df)):
    
    df['match_centroid_area'][i] = df2['centroid'][df['nearest_matching_polygon_wrt_area_ts_ss'][i]]
    df['match_centroid_dist'][i] = df2['centroid'][df['nearest_matching_polygon_wrt_dist_ts_ss'][i]]
    df['match_centroid_ecc'][i] = df2['centroid'][df['nearest_matching_polygon_wrt_ecc_ts_ss'][i]]
    df['match_centroid_overall'][i] = df2['centroid'][df['nearest_matching_polygon_overall_ts_ss'][i]]

In [ ]:
from shapely.geometry import Point, LineString
df['line_ecc'] = df.apply(lambda x: LineString([x['centroid'], x['match_centroid_ecc']]), axis=1)
df['line_area'] = df.apply(lambda x: LineString([x['centroid'], x['match_centroid_area']]), axis=1)
df['line_dist'] = df.apply(lambda x: LineString([x['centroid'], x['match_centroid_dist']]), axis=1)
df['line_overall'] = df.apply(lambda x: LineString([x['centroid'], x['match_centroid_overall']]), axis=1)

In [ ]:
list(df.columns)

In [ ]:
df = df.set_geometry('line')

In [ ]:
for i in range(0, 500, 5):
    #print(i)

    fig, ax = plt.subplots(figsize = (16,16))
    df_comb.plot(color = df_comb['color'], ax=ax)
    df[i:i+5].plot(ax=ax)
    #df[100:105].plot(ax=ax)

    fig.savefig('output/img' + str(i) + '.png')

In [ ]:
from matplotlib.patches import ConnectionPatch

for j in range(0, 7000, 1000): 
    fig= plt.figure(figsize = (40,20))
    fig.patch.set_facecolor('white')

    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122, sharey=ax1)
    ax1.set_title('Aug 2020')
    ax2.set_title('Oct 2020')
    
    df = df.set_geometry('geometry')
    df2.plot(color = 'g', ax=ax1)
    df.plot(color = 'r', ax=ax2)

    for i in range(j, j+1000, 10):
        #i = 10
        xy = [y for y in df['line_ecc'][i].coords]
        #xy = (x[i],y[i])
        con = ConnectionPatch(xyA=xy[1], xyB=xy[0], coordsA="data", coordsB="data",
                              axesA=ax1, axesB=ax2, color="blue")
        ax2.add_artist(con)

        ax1.plot(xy[1][0],xy[1][1],'ro',markersize=10)
        ax2.plot(xy[0][0],xy[0][1],'ro',markersize=10)

    fig.savefig('output/img' + str(j) + '.png')
#     plt.show()

In [ ]:
import cv2
import numpy as np
import os
import glob

image_folder = r"output/"
video_file = "matchings_3fps_ts_ss_10_ecc.mp4"
# image_size = (160, 120)
fps = 3

#images = [img for img in sorted(glob.glob(image_folder))]
images = [img for img in sorted(os.listdir(image_folder)) if '.png' in img]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape
image_size = (width,height)

out = cv2.VideoWriter(video_file, cv2.VideoWriter_fourcc(*'MP4V'), fps, image_size)

img_array = []
for filename in images:
    img = cv2.imread(os.path.join(image_folder, filename))
    img_array.append(img)
    out.write(img)

out.release()

In [ ]:
df.head()

In [ ]:
list(df.columns)

In [ ]:
df_sorted = df.sort_values('min_ts_ss_area')
df_sorted.reset_index(drop=True, inplace=True)
df_sorted.head()

In [ ]:
fig, ax = plt.subplots(figsize = (20,20))
fig.patch.set_facecolor('white')


df2.plot(color = 'g', ax=ax, figsize=(20,20))
df.plot(color = 'r', ax=ax , figsize=(20,20))

for i in range(0, 10):
        #i = 10
        xy = [y for y in df_sorted['line_area'][i].coords]
        #xy = (x[i],y[i])
        con = ConnectionPatch(xyA=xy[1], xyB=xy[0], coordsA="data", coordsB="data",
                              axesA=ax, axesB=ax, color="blue")
        ax.add_artist(con)

        ax.plot(xy[1][0],xy[1][1],'ro',markersize=10, color='g')
        ax.plot(xy[0][0],xy[0][1],'ro',markersize=10, color='r')

fig.show()

In [ ]:
print(df_sorted['line_ecc'][0])

In [ ]:
df_sorted['centroid'][0].x